## Version 0.9.9 2021 03 08

### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 The code is also hosted at https://github.com/n4rus/tbot and is licensed under creative commons.
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [1]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import datetime

### API token, secret key and acess keys
API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [2]:
from secrets import SecretKey
from secrets import AccessKey
from novadax import RequestClient as NovaClient
from datetime import datetime

nova_client = NovaClient(AccessKey, SecretKey)

### Importing List of currencies, making the API Call and the table Dataframe
List of crypto-currencies, get balance, and table creation

In [3]:
balance = nova_client.get_account_balance()
bdata = balance['data']

bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0
bcr = {}

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bcr[f"{bic}"] = f"{bis}"   
    bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
    bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)
    bs = bs + 1  

sublcq = nova_client.subs()
sublcd = sublcq['data']

blccolumns = ['Currency', 'Sub-Balance']
blcdataframe = pd.DataFrame(columns = bcolumns,)

p = 0
subids = {}
sublcs = {}

for i in sublcd:
    sublcd = sublcq['data']
    sub = sublcd[p]
    sublci = sub['subId']
    sublcy = sub['subIdentify']
    subids[f"{sublcy}"] = f"{sublci}"
    subId = sublci
    sublcid = nova_client.subs_balance(subId)
    subd = sublcid['data']
    sd = subd[p]
    subcy = sd['currency']
    sublc = sd['balance']
    sublcs[f"{subcy}"] = f"{sublc}"
    
    blcdataframe.append(
        pd.Series([
            subcy,
            sublc,
        ], index = blccolumns),
        ignore_index = True)
    blcdataframe = blcdataframe.append(
    pd.Series([
        subcy,
        sublc,
    ], index=blccolumns),
        ignore_index=True)
 
    p = p + 1

blcdataframe.sort_values('Balance', ascending = False, inplace = True)
blcdataframe.reset_index(drop = True, inplace = True)

bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
#bdataframe

In [4]:
blcdataframe.sort_values('Sub-Balance', ascending = False, inplace = True)
blcdataframe.reset_index(drop = True, inplace = True)
#blcdataframe

### Symbol position, Dolar price and timestamp
For the execution to be done independently for each symbol, the below cell is needed:

In [34]:
symbol = 'ETH_USDT'
symbols = ['ETH_USDT']
currency = 'R$'
my_columns = ['Symbol', 'Currency Price', 'Number of Shares to Buy']
e_dataframe = pd.DataFrame(columns = my_columns)

spld = nova_client.list_tickers()

for symbol in symbols:
    t = 0
    for i in spld['data']:  
        fdata = spld['data']
        fdt = fdata[t]
        price = fdt['price']
        amount = fdt['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
        etsp = (fdt['timestamp'])
        r = r + 1
        e_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        shares_to_buy
                    ]
                    ,index = my_columns),
                ignore_index=True)
    
    
        e_dataframe = e_dataframe.append(
            pd.Series([
                        symbol, 
                        price,
                        shares_to_buy
                        ],
                    index = my_columns
                    ),
                    ignore_index = True)

e_dataframe

KeyError: 'price'

### Creating the momentum table
For mommentum we use the last order minus the order from an amount x of time. The place from wich the data comes is main importance on the formula. As stated in the comment section on the first versions, data must come from the common filled orders or the last trades from the users history, here the second case is uded, as well from this version on, but the first ones used common filled orders.

In [26]:
listtrades = nova_client.list_orders(symbol, 'FILLED', limit=5)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
my_columns = ['Symbol', 'Amount', 'Number of Shares to Buy','Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data']
m_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listtrades['data'][:9]:
        ldata = listtrades['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        sharestobuy = float(amount) - (float(edolarwal))/float(price)
        bs = ldc['side']
        order = float(amount) - float(sharestobuy)
        srto = float(order)
        
        #M = V - Vx ; M representa o indicador de momentum, V o valor de fechamento atual e V(x_diasatras) o valor anterior
        #nao é deste modo que se utiliza o simbolo x na multiplicação. Vx seriam multiplos valores de V em x dias. Para este caso, Vi
        #veja que por conotaçao convencional parece nao fazer diferença, porém, na matemática, tais convenções são necessárias e de certo modo indispensáveis no tempo t
        #h = 
        vxdata = e_dataframe['Currency Price']
        vxi = vxdata[s]
        vx = vxi
        momentum = float(price) - float(vx)
        tsp = (ldc['timestamp'])
        s = s + 1
        m_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        m_dataframe = m_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

m_dataframe.sort_values('Last Orders Price', ascending = True, inplace = True)
m_dataframe.reset_index(drop = True, inplace = True)
m_dataframe

,Symbol,Amount,Number of Shares to Buy,Last Orders Price,B/S,Momentum Price,momentum-raw-data
0,ETH_USDT,0.0697,0.000058,1804.33,BUY,1796.29,8.04
1,ETH_USDT,0.1413,0.071659,1816.89,SELL,1796.29,20.60
2,ETH_USDT,0.1406,0.070959,1825.00,BUY,1796.29,28.71
3,ETH_USDT,0.1413,0.071659,1827.14,BUY,1796.29,30.85
4,ETH_USDT,0.0391,-0.030540,1830.85,SELL,1796.29,34.56
5,ETH_USDT,0.1395,0.069860,1838.47,BUY,1796.29,42.18
6,ETH_USDT,0.1118,0.042160,1839.31,SELL,1796.29,43.02
7,ETH_USDT,0.1405,0.070860,1840.00,SELL,1796.29,43.71
8,ETH_USDT,0.0391,-0.030540,1850.77,SELL,1796.29,54.48


### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin. Here as columns the last used titles shall be all used in one go to give a more complete dataframe in our xlsx database.

In [7]:
#orders dataframe
ttrades = nova_client.list_trades(symbol, limit=9)
tdata = ttrades['data']
my_columns = ['Symbol', 'Amount', 'Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data', 'Status', 'Timestamp']
o_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listorders = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listorders['data'][:9]:
        ldata = listorders['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        bs = ldc['side']
        
        #rework into a precise formula with momentum
        #This must free the mommentum table cell
        vxdata = tdata[s]
        vxi = vxdata['price']
        momentumprice = float(vxi)
        momentumraw = float(price) - float(vxi)

        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        s = s + 1
        
        o_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        o_dataframe = o_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

o_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
o_dataframe.reset_index(drop = True, inplace = True)
o_dataframe

,Symbol,Amount,Last Orders Price,B/S,Momentum Price,momentum-raw-data,Status,Timestamp
0,ETH_USDT,0.0697,1804.33,BUY,1792.85,11.48,FILLED,20210310 17:46:17
1,ETH_USDT,0.0391,1850.77,SELL,1777.94,72.83,FILLED,20210310 16:17:31
2,ETH_USDT,0.0391,1830.85,SELL,1811.39,19.46,CANCELED,20210310 13:22:47
3,ETH_USDT,0.1118,1839.31,SELL,1810.84,28.47,CANCELED,20210310 13:15:18
4,ETH_USDT,0.1413,1827.14,BUY,1810.84,16.30,PARTIAL_CANCELED,20210310 12:53:09
5,ETH_USDT,0.1405,1840.00,SELL,1804.66,35.34,FILLED,20210309 11:50:06
6,ETH_USDT,0.1406,1825.00,BUY,1800.00,25.00,FILLED,20210308 23:10:58
7,ETH_USDT,0.1395,1838.47,BUY,1825.09,13.38,CANCELED,20210308 22:51:25
8,ETH_USDT,0.1413,1816.89,SELL,1857.82,-40.93,FILLED,20210308 21:30:09


In [8]:
#sort price [ok]
#wallet_amount [ok]
#dataframe momentum[ok]pt2[ok]
#buy lower(wallet_amount - amount)  [ok]   
#sell + profit(constant*market momentum)[ok]


#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}

#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#20200225#v0-9-1# the data must come from the common filled orders or the last trades from the users history, 
# here the second case is uded, as well from this version on, first ones used common filled orders

#loop time count and do momentum profit auto cicle
#publish

listorders = nova_client.list_orders(symbol, 'FINISHED')
for i in o_dataframe['Last Orders Price']:
#define booleans
#boolean dataframe
    a = 9 
    b = 2
    gama = 0
    z = 0
    sti = 0
    filled = 'FILLED'
    statushd = {}
    stdata = o_dataframe['Status']
    bsdata = o_dataframe['B/S']
    lopdta = o_dataframe['Last Orders Price']

for i in bsdata:
        bsdaty = bsdata[z]
        stdaty = stdata[z] 
        lopdty = lopdta[z]
        statushd[f"{bsdaty}"] = f"{stdaty}, {lopdty}"
        z = z + 1
print(statushd)

{'BUY': 'CANCELED, 1838.47', 'SELL': 'FILLED, 1816.89'}


In [27]:

my_columns = ['B/S', 'Status','Last Orders Price', 'Timestamp']
boolean_dataframe = pd.DataFrame(columns = my_columns)


listorders = nova_client.list_orders(symbol, 'FINISHED')
for i in o_dataframe['Last Orders Price']:
#define booleans
#boolean dataframe
    a = 9 
    b = 2
    gama = 0
    z = 0
    sti = 0
    filled = 'FILLED'
    statushd = {}
    stdata = o_dataframe['Status']
    bsdata = o_dataframe['B/S']
    lopdta = o_dataframe['Last Orders Price']
    for i in stdata:
        bsdaty = bsdata[z]
        stdaty = stdata[z] 
        lopdty = lopdta[z]
        ldata = listorders['data']
        ldc = ldata[z]
        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        sell = float(lopdty) - float(lopdty *(0.03))    
        buy = float(lopdty) - float(lopdty *(0.03))
        profit = sell - buy
        bm = buy - momentumprice
        if stdaty == filled:
            statushd[f"{bsdaty}"] = f"{stdaty}, {lopdty}"
            boolean_dataframe.append(
                    pd.Series(
                    [
                        bsdaty,
                        stdaty,
                        lopdty,
                        tsp
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        boolean_dataframe = boolean_dataframe.append(
                pd.Series([
                        bsdaty,
                        stdaty,
                        lopdty,
                        tsp
                      ]
                ,index = my_columns), 
                ignore_index = True)

boolean_dataframe
if momentumprice > price and amount > 0.02 :
    sellprofit = math.floor(price*2/100)/1.00
    result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = profit, amount = amount)  

#first block to be solved
if momentumprice < price and amount < 0.02:
    print("Market is not ready to sell")

if momentumprice < price and amount > 0.02:
    result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = bmprice, amount = amount)
    print("The quantity bought was ETH: ", bm)
    

RequestException: ('A30012', "Price can't be less than 0.9 of current price")

In [28]:
sellprofit = math.floor(price*2/100)/1.00
spe = price + sellprofit
print(spe)

1852.89


In [ ]:
time = d.ctime()
print(datetime.fromtimestamp(time.time()))

### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [ ]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
o_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Amount', float_template],
    'C':['Price', dollar_template],
}

for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [ ]:
writer.save()